In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import metrics   

from sklearn import set_config
set_config(display='diagram') # Useful for display the pipeline

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#dataset_half_S = pd.read_csv('Datasets/dataset_halfSecondWindow.csv', index_col=[0])
dataset = pd.read_csv('Datasets/dataset_5SecondWindow.csv')

In [4]:
dataset.head(10)

,Unnamed: 0,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.000000,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.000000,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.00,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.000000,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.000000,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.000000,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.000000,0

In [5]:
dataset.user.unique().tolist() #all users

['U12',
 'U1',
 'U8',
 'U7',
 'U10',
 'U13',
 'U6',
 'U11',
 'U3',
 'U2',
 'U4',
 'U9',
 'U5']

In [6]:
for name, group in dataset.groupby('user'):
    print(name,len(group)) 

U1 2470
U10 531
U11 167
U12 820
U13 90
U2 271
U3 331
U4 227
U5 17
U6 279
U7 397
U8 214
U9 79


In [7]:
dataset.target.unique().tolist() #all users

['Still', 'Car', 'Train', 'Bus', 'Walking']

In [8]:
#dataset.target = dataset.target.replace({'Car':'Vehicle','Bus':'Vehicle'})

In [9]:
for name, group in dataset.groupby('target'):
    print(name,len(group)) 

Bus 1178
Car 1180
Still 1179
Train 1179
Walking 1177


In [10]:
# encode the target
#dataset.target = dataset.target.replace({'Still':0, 'Walking':1, 'Car':2, 'Bus':3, 'Train':4})

In [11]:
#dataset.columns

id
time
activityrecognition # 2
android.sensor.accelerometer #mmms //Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), including the force of gravityacceleration force
android.sensor.game_rotation_vector # mmms //orientation of the device
android.sensor.gravity #mmms //Measures the force of gravity in m/s2 that is applied to a device on all three physical axes (x, y, z)
android.sensor.gyroscope #mmms //Measures a device's rate of rotation in rad/s around each of the three physical axes (x, y, and z)
android.sensor.light #mmms //Measures the ambient light level (illumination) in lx
android.sensor.linear_acceleration #mmms // Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), excluding the force of gravity
android.sensor.magnetic_field #mmms //Measures the ambient geomagnetic field for all three physical axes (x, y, z) in μT
android.sensor.magnetic_field_uncalibrated #mmms
android.sensor.orientation #mmms
android.sensor.pressure #mmms //Measures the ambient air pressure in hPa or mbar.
android.sensor.proximity #mmms //Measures the proximity of an object in cm relative to the view screen of a device. This sensor is typically used to determine whether a handset is being held up to a person's ear.
android.sensor.rotation_vector #mmms //Measures the orientation of a device by providing the three elements of the device's rotation vector.
android.sensor.step_counter #mmms
sound #mmms
speed #mmms
target
user

targets : walking, car, still, train and bus. 
26% of data is annoted as walking, 25% as driving a car, 24% as standing still, 20% as being on train, and 5% as being on bus. 

In [12]:
#dataset.info()

In [13]:
dataset.describe()

,Unnamed: 0,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std
count,5893.0000,5893.000000,5893.000000,0.0,5893.000000,5893.000000,5893.000000,5893.000000,5893.000000,5094.000000,5094.000000,5094.000000,5093.000000,5396.000000,5396.000000,5396.000000,5.356000e+03,5173.000000,5173.000000,5173.000000,5172.000000,5094.000000,5094.000000,5094.000000,5094.000000,4381.000000,4381.000000,4381.000000,2124.000000,5399.000000,5399.000000,5399.000000,5370.000000,5400.000000,5400.000000,5400.000000,5383.000000,5094.000000,5094.000000,5094.000000,5087.000000,5396.000000,5396.000000,5396.000000,5373.000000,1918.000000,1918.000000,1918.000000,1128.000000,1119.000000,1119.000000,1119.000000,283.000000,5172.000000,5172.000000,5172.000000,5163.000000,1487.000000,1487.000000,1487.000000,232.000000,4899.000000,4899.000000,4899.000000,4062.000000,3520.000000,3520.000000,3520.000000,1318.000000
mean,2946.0000,10685.407263,117.731716,NaN,87.718310,10.007976,8.426262,12.247691,0.982987,0.759280,0.743872,0.771629,0.011876,9.806561,9.806530,9.806592,1.220395e-04,0.312536,0.089494,0.735042,0.189903,0.312807,0.125427,0.614961,0.174279,288.825545,254.140861,331.698065,85.357327,1.315971,0.863086,2.157592,0.602771,56.212508,54.267805,58.207109,1.772942,138.409066,136.955896,139.907248,2.038359,212.629637,199.930476,224.731494,12.488254,1016.203139,1016.189143,1016.218816,0.023097,6.117128,5.174263,6.797140,4.081700,0.765932,0.750273,0.779000,0.012977,10112.523523,10112.205783,10112.769334,2.959475,78.724855,78.451658,78.997213,1.913065,8.235254,8.216747,8.253528,0.236959
std,1701.3069,6921.229833,126.474151,NaN,24.562728,0.655949,1.783007,5.178436,1.505615,0.251174,0.255232,0.251401,0.026647,0.001610,0.001709,0.001584,7.681037e-04,0.554585,0.206327,1.356240,0.342901,0.537925,0.235223,1.112451,0.322397,1761.573894,1361.427348,2379.942459,1052.057842,2.065444,1.493150,5.058223,1.136752,38.210015,37.902702,38.681821,2.938644,71.034674,70.764992,71.910605,5.423627,95.253318,98.160410,101.113269,31.517931,14.142150,14.

In [14]:
dataset.isnull().sum()

Unnamed: 0                                            0
id                                                    0
time                                                  0
activityrecognition#0                              5893
activityrecognition#1                                 0
android.sensor.accelerometer#mean                     0
android.sensor.accelerometer#min                      0
android.sensor.accelerometer#max                      0
android.sensor.accelerometer#std                      0
android.sensor.game_rotation_vector#mean            799
android.sensor.game_rotation_vector#min             799
android.sensor.game_rotation_vector#max             799
android.sensor.game_rotation_vector#std             800
android.sensor.gravity#mean                         497
android.sensor.gravity#min                          497
android.sensor.gravity#max                          497
android.sensor.gravity#std                          537
android.sensor.gyroscope#mean                   

In [15]:
# replace anroid.sensor & #
dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')
dataset.head()

<ipython-input-15-e92dca201964>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')


,Unnamed: 0,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,game_rotation_vector_max,game_rotation_vector_std,gravity_mean,gravity_min,gravity_max,gravity_std,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_std,gyroscope_uncalibrated_mean,gyroscope_uncalibrated_min,gyroscope_uncalibrated_max,gyroscope_uncalibrated_std,light_mean,light_min,light_max,light_std,linear_acceleration_mean,linear_acceleration_min,linear_acceleration_max,linear_acceleration_std,magnetic_field_mean,magnetic_field_min,magnetic_field_max,magnetic_field_std,magnetic_field_uncalibrated_mean,magnetic_field_uncalibrated_min,magnetic_field_uncalibrated_max,magnetic_field_uncalibrated_std,orientation_mean,orientation_min,orientation_max,orientation_std,pressure_mean,pressure_min,pressure_max,pressure_std,proximity_mean,proximity_min,proximity_max,proximity_std,rotation_vector_mean,rotation_vector_min,rotation_vector_max,rotation_vector_std,step_counter_mean,step_counter_min,step_counter_max,step_counter_std,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.0,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.0,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.0,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.0,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.0,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.0,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.0,0.0,0.0,NaN,1.488361,0.575910,2.415879,0.920077,37.836471,37.525725,38.147217,0.439461,95.743821,95.664309,95.823332,0.112446,373.556039,359.897250,397.071741,8.870011,1006.950440,1006.95044,1006.95044,NaN,NaN,NaN,NaN,NaN,0.998112,0.998112,0.998112,0.002788,NaN,NaN,NaN,NaN,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,U12
4,4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,0.672664,0.064792,9.806650,9.806650,9.806651,2.644430e-07,0.183202,0.020667,0.380875,0.098819,0.265652,0.141208,0.313020,0.083175,30.0,30.0,30.0,39.597980,1.658308,0.379803,2.795995,1.122225,78.375718,76.543261,79.273739,1.104361,157.429218,156.795909,158.062526,0.895633,193.201458,35.864972,360.560468,181.729455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.322242,0.308055,0.378193,0.025608,NaN,NaN,NaN,NaN,89.770732,89.770732,89.770732,0.006

In [16]:
dataset['rv_gyro_mean'] = dataset['rotation_vector_mean'] * dataset['gyroscope_mean']
dataset['lin_speed_mean'] = dataset['linear_acceleration_mean'] * dataset['speed_mean']
dataset['rv_gyro_prox_mean'] = dataset['rotation_vector_mean'] * dataset['gyroscope_mean']* dataset['proximity_std']
dataset['lin_speed_prox_mean'] = dataset['linear_acceleration_mean'] * dataset['speed_mean'] *dataset['proximity_std']
dataset['rv_gyro__grv_mean'] = dataset['rotation_vector_mean'] * dataset['gyroscope_mean']* dataset['game_rotation_vector_mean']

In [17]:
dataset = dataset[(['accelerometer_min',
                    'accelerometer_max',
                    'accelerometer_std',
                    'gyroscope_mean',
                    'gyroscope_uncalibrated_mean',
                    'gyroscope_std',
                    'linear_acceleration_mean',
                    'rotation_vector_mean',
                    'rotation_vector_max',
                    'proximity_std',
                    'magnetic_field_uncalibrated_min',
                    'sound_min',
                    'sound_max',
                    'sound_mean',
                    
                    'rv_gyro_mean',
                    
                    'lin_speed_mean',
                    'rv_gyro_prox_mean',
                    'lin_speed_prox_mean',
                    'rv_gyro__grv_mean',
                    
                    'user',
                    'target'
                 ])]

In [18]:
dataset.columns

Index(['accelerometer_min', 'accelerometer_max', 'accelerometer_std',
       'gyroscope_mean', 'gyroscope_uncalibrated_mean', 'gyroscope_std',
       'linear_acceleration_mean', 'rotation_vector_mean',
       'rotation_vector_max', 'proximity_std',
       'magnetic_field_uncalibrated_min', 'sound_min', 'sound_max',
       'sound_mean', 'rv_gyro_mean', 'lin_speed_mean', 'rv_gyro_prox_mean',
       'lin_speed_prox_mean', 'rv_gyro__grv_mean', 'user', 'target'],
      dtype='object')

In [19]:
dataset.head()

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,gyroscope_std,linear_acceleration_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,sound_max,sound_mean,rv_gyro_mean,lin_speed_mean,rv_gyro_prox_mean,lin_speed_prox_mean,rv_gyro__grv_mean,user,target
0,9.758895,9.849411,0.014626,0.001651,0.016221,0.000737,0.020978,0.050413,0.056351,NaN,51.199707,NaN,NaN,NaN,0.000083,0.000000,NaN,NaN,0.000002,U12,Still
1,7.707437,17.146631,1.775944,0.036326,0.039023,0.020290,0.879220,0.999981,0.999999,NaN,82.409890,89.065143,89.335277,89.200210,0.036325,14.541729,NaN,NaN,0.036322,U12,Car
2,9.804817,9.849262,0.011199,0.001525,0.016302,0.000713,0.034110,0.610456,0.610456,NaN,55.501802,NaN,NaN,NaN,0.000931,0.000000,NaN,NaN,0.000619,U12,Still
3,7.659674,12.304298,0.862553,0.036440,0.050759,0.010553,1.488361,0.998112,0.998112,NaN,95.664309,87.470377,87.470377,87.470377,0.036372,26.403360,NaN,NaN,0.036234,U12,Car
4,8.965621,10.891645,0.504117,0.183202,0.265652,0.098819,1.658308,0.322242,0.378193,NaN,156.795909,89.770732,89.770732,89.770732,0.059035,14.924768,NaN,NaN,0.033284,U1,Car


In [20]:
# split user = ['Luca' 'Pierpaolo' 'IvanHeibi']
dataset_test = dataset[(dataset['user'] == 'U2')|(dataset['user'] == 'U3')|(dataset['user'] == 'U4')]
dataset_train = dataset[(dataset['user'] != 'U2')&(dataset['user'] != 'U3')&(dataset['user'] != 'U4')]
print(len(dataset))
print(len(dataset_test))
print(len(dataset_train))

5893
829
5064


In [21]:
dataset_train.sample(10)

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,gyroscope_std,linear_acceleration_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,sound_max,sound_mean,rv_gyro_mean,lin_speed_mean,rv_gyro_prox_mean,lin_speed_prox_mean,rv_gyro__grv_mean,user,target
5233,9.454947,10.383413,0.286640,0.056027,0.046280,0.016375,0.991837,0.838653,0.863058,NaN,83.577869,77.231883,85.539800,81.385841,0.046987,12.104333,NaN,NaN,0.028246,U11,Car
3035,6.100275,21.379260,4.550347,1.512945,1.836613,0.942724,2.720083,0.793503,0.840531,NaN,115.741639,89.758879,89.758879,89.758879,1.200527,3.060093,NaN,NaN,1.089383,U1,Walking
4269,10.173328,10.234160,0.021083,0.041750,0.051968,0.054182,0.039345,0.994845,0.995783,NaN,100.792935,51.077661,51.077661,51.077661,0.041534,NaN,NaN,NaN,0.041386,U1,Still
5642,8.697494,11.111561,0.675826,0.196971,0.126271,0.093472,0.755883,0.960453,0.976213,NaN,150.092604,89.749555,89.749555,89.749555,0.189182,8.314710,NaN,NaN,0.152245,U1,Bus
4062,9.711212,9.766960,0.017716,0.003224,0.041539,0.001121,0.061859,0.221378,0.221378,NaN,168.920662,68.815034,68.815034,68.815034,0.000714,NaN,NaN,NaN,0.000136,U7,Train
1400,9.322562,9.725353,0.108727,0.020978,0.037640,0.012366,0.157318,0.876836,0.880364,NaN,152.619474,89.785948,89.785948,89.785948,0.018395,1.061893,NaN,NaN,0.011269,U1,Bus
3387,9.771079,9.874825,0.014621,0.001784,0.016276,0.000616,0.024062,0.061018,0.066371,NaN,50.619759,NaN,NaN,NaN,0.000109,0.000000,NaN,NaN,0.000011,U12,Still
3543,8.520991,10.780693,0.574361,0.079499,0.089448,0.049966,0.567366,0.911879,0.915072,NaN,390.327015,89.769040,89.769040,89.769040,0.072493,17.446499,NaN,NaN,0.061370,U1,Car
955,9.379497,11.311392,0.647284,0.236045,0.296419,0.142811,1.122057,0.465317,0.477820,NaN,132.440471,NaN,NaN,NaN,0.109836,NaN,NaN,NaN,0.101120,U1,Bus
213,8.784329,10.435172,0.337545,0.030069,0.044158,0.013210,0.188880,0.608824,0.608824,NaN,70.857111,75.335690,75.335690,75.335690,0.018307,1.229847,NaN,NaN,0.013514,U11,Car


In [22]:
#dataset_train = dataset_train.fillna(0)

In [23]:
dataset_test.head()

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,gyroscope_std,linear_acceleration_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,sound_max,sound_mean,rv_gyro_mean,lin_speed_mean,rv_gyro_prox_mean,lin_speed_prox_mean,rv_gyro__grv_mean,user,target
33,9.764513,14.028126,1.283204,0.615399,0.778458,0.655962,1.721314,0.955202,0.955202,NaN,36.378866,77.064227,77.064227,77.064227,0.587830,2.050172,NaN,NaN,0.554815,U3,Walking
49,3.324387,14.608536,4.031835,0.597423,0.619693,0.167046,4.223926,0.934642,0.937574,NaN,87.072128,77.221125,77.221125,77.221125,0.558377,NaN,NaN,NaN,0.555901,U2,Walking
51,4.595131,17.636696,3.551271,NaN,NaN,NaN,2.791507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U4,Walking
54,8.712177,9.963264,0.361961,0.447802,0.334574,0.116564,0.958200,0.983561,0.989576,NaN,77.496043,74.178611,74.178611,74.178611,0.440440,0.000000,NaN,NaN,0.393798,U3,Bus
56,5.371977,15.088214,3.438894,0.550989,0.749501,0.250989,3.610910,0.802652,0.823187,NaN,72.877824,78.588379,80.072657,79.330518,0.442252,4.333092,NaN,NaN,0.441829,U2,Walking


In [24]:
# drop user in test & split
dataset_train = dataset_train.drop(['user'], axis = 1)
dataset_test = dataset_test.drop(['user'], axis = 1)

In [45]:
num_vars = ['accelerometer_min',
            'accelerometer_max',
            'accelerometer_std',
            'gyroscope_mean',
            'gyroscope_uncalibrated_mean',
            'gyroscope_std',
            
            'linear_acceleration_mean',
            
            'rotation_vector_mean',
            'rotation_vector_max',
            'proximity_std',
            'magnetic_field_uncalibrated_min',
            'sound_min',
            'sound_max',
            'sound_mean',
            
            
            #'rv_gyro_mean',
            #'lin_speed_mean',
            #'rv_gyro_prox_mean',
            #'lin_speed_prox_mean',
            #'rv_gyro__grv_mean',
            ]


In [46]:
# YOUR CODE HERE
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.KNNImputer(n_neighbors=3)),
  ('Normalizer',RobustScaler())
  
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars)
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=3)),
                                                 ('Normalizer',
                                                  RobustScaler())]),
                                 ['accelerometer_min', 'accelerometer_max',
                                  'accelerometer_std', 'gyroscope_mean',
                                  'gyroscope_uncalibrated_mean',
                                  'gyroscope_std', 'linear_acceleration_mean',
                                  'rotation_vector_mean', 'rotation_vector_max',
                                  'proximity_std',
                                  'magnetic_field_uncalibrated_min',
                                  'sound_min', 'sound_max', 'sound_mean'])])

In [47]:


from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier



In [48]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}



tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=3)),
                                                                  ('Normalizer',
                                                                   RobustScaler())]),
                                                  ['accelerometer_min',
                                                   'accelerometer_max',
                                                   'accelerometer_std',
                                                   'gyroscope_mean',
                                                   'gyroscope_uncalibrated_mean',
                                                   'gyroscope_std',
                                                   'linear_acceleration_mean',
                                                   'rotation_vector_mean',
                                                   'rotation_vector_max',
                                                   'proximity_std',
                                                   'magnetic_field_uncalibrated_min',
                                                   'sound_min', 'sound_max',
                                                   'sound_mean'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [49]:
# split 
x_train = dataset_train.drop(['target'], axis=1)
y_train = dataset_train.target
x_test = dataset_test.drop(['target'], axis=1)
y_test = dataset_test.target

In [50]:
x_train.columns

Index(['accelerometer_min', 'accelerometer_max', 'accelerometer_std',
       'gyroscope_mean', 'gyroscope_uncalibrated_mean', 'gyroscope_std',
       'linear_acceleration_mean', 'rotation_vector_mean',
       'rotation_vector_max', 'proximity_std',
       'magnetic_field_uncalibrated_min', 'sound_min', 'sound_max',
       'sound_mean', 'rv_gyro_mean', 'lin_speed_mean', 'rv_gyro_prox_mean',
       'lin_speed_prox_mean', 'rv_gyro__grv_mean'],
      dtype='object')

In [51]:
from sklearn import model_selection
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x_train, y_train,
    test_size=0.2,
    stratify = y_train,   
    random_state=37
)

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_test)  
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:06:55] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.084978
0:	learn: 1.4873106	total: 33.2ms	remaining: 33.2s
1:	learn: 1.4013423	total: 53.5ms	remaining: 26.7s
2:	learn: 1.3171740	total: 72.8ms	remaining: 24.2s
3:	learn: 1.2490382	total: 96.2ms	remaining: 24s
4:	learn: 1.1923226	total: 121ms	remaining: 24.1s
5:	learn: 1.1426116	total: 138ms	remaining: 22.9s
6:	learn: 1.1021250	total: 166ms	remaining: 23.5s
7:	learn: 1.0587628	total: 187ms	remaining: 23.2s
8:	learn: 1.0228766	total: 222ms	remaining: 24.4s
9:	learn: 0.9841900	total: 260ms	remaining: 25.7s
10:	learn: 0.9586406	total: 291ms	remaining: 26.2s
11:	learn: 0.9322279	total: 322ms	remaining: 26.5s
12:	learn: 0.9054697	total: 345ms	remaining: 26.2s
13:	learn: 0.8834460	total: 383ms	remaining: 27s
14:	learn: 0.

159:	learn: 0.2854406	total: 6.44s	remaining: 33.8s
160:	learn: 0.2848716	total: 6.47s	remaining: 33.7s
161:	learn: 0.2846560	total: 6.5s	remaining: 33.6s
162:	learn: 0.2838671	total: 6.54s	remaining: 33.6s
163:	learn: 0.2828774	total: 6.57s	remaining: 33.5s
164:	learn: 0.2822642	total: 6.58s	remaining: 33.3s
165:	learn: 0.2816581	total: 6.61s	remaining: 33.2s
166:	learn: 0.2808385	total: 6.63s	remaining: 33.1s
167:	learn: 0.2800060	total: 6.66s	remaining: 33s
168:	learn: 0.2792391	total: 6.68s	remaining: 32.9s
169:	learn: 0.2781224	total: 6.71s	remaining: 32.7s
170:	learn: 0.2768219	total: 6.74s	remaining: 32.7s
171:	learn: 0.2751543	total: 6.77s	remaining: 32.6s
172:	learn: 0.2742799	total: 6.8s	remaining: 32.5s
173:	learn: 0.2730078	total: 6.82s	remaining: 32.4s
174:	learn: 0.2719430	total: 6.83s	remaining: 32.2s
175:	learn: 0.2713788	total: 6.86s	remaining: 32.1s
176:	learn: 0.2706278	total: 6.89s	remaining: 32s
177:	learn: 0.2699076	total: 6.92s	remaining: 32s
178:	learn: 0.269175

321:	learn: 0.1861361	total: 11.5s	remaining: 24.2s
322:	learn: 0.1857283	total: 11.5s	remaining: 24.1s
323:	learn: 0.1853960	total: 11.5s	remaining: 24.1s
324:	learn: 0.1849978	total: 11.6s	remaining: 24s
325:	learn: 0.1846060	total: 11.6s	remaining: 23.9s
326:	learn: 0.1841876	total: 11.6s	remaining: 23.9s
327:	learn: 0.1835101	total: 11.6s	remaining: 23.8s
328:	learn: 0.1833137	total: 11.7s	remaining: 23.8s
329:	learn: 0.1828933	total: 11.7s	remaining: 23.7s
330:	learn: 0.1822335	total: 11.7s	remaining: 23.7s
331:	learn: 0.1816519	total: 11.8s	remaining: 23.6s
332:	learn: 0.1811667	total: 11.8s	remaining: 23.6s
333:	learn: 0.1807444	total: 11.8s	remaining: 23.5s
334:	learn: 0.1804224	total: 11.8s	remaining: 23.5s
335:	learn: 0.1800211	total: 11.8s	remaining: 23.4s
336:	learn: 0.1797092	total: 11.9s	remaining: 23.3s
337:	learn: 0.1791966	total: 12s	remaining: 23.6s
338:	learn: 0.1789882	total: 12.1s	remaining: 23.5s
339:	learn: 0.1785151	total: 12.1s	remaining: 23.5s
340:	learn: 0.17

481:	learn: 0.1353316	total: 17.4s	remaining: 18.7s
482:	learn: 0.1350608	total: 17.6s	remaining: 18.9s
483:	learn: 0.1346687	total: 17.8s	remaining: 19s
484:	learn: 0.1343284	total: 17.9s	remaining: 19s
485:	learn: 0.1341136	total: 18s	remaining: 19s
486:	learn: 0.1338818	total: 18s	remaining: 19s
487:	learn: 0.1335219	total: 18s	remaining: 18.9s
488:	learn: 0.1333807	total: 18.1s	remaining: 18.9s
489:	learn: 0.1332199	total: 18.1s	remaining: 18.9s
490:	learn: 0.1330844	total: 18.2s	remaining: 18.9s
491:	learn: 0.1329570	total: 18.2s	remaining: 18.8s
492:	learn: 0.1327447	total: 18.3s	remaining: 18.8s
493:	learn: 0.1324815	total: 18.4s	remaining: 18.8s
494:	learn: 0.1321736	total: 18.5s	remaining: 18.9s
495:	learn: 0.1319162	total: 18.6s	remaining: 18.9s
496:	learn: 0.1316278	total: 18.7s	remaining: 18.9s
497:	learn: 0.1315228	total: 18.8s	remaining: 18.9s
498:	learn: 0.1313072	total: 18.8s	remaining: 18.9s
499:	learn: 0.1311236	total: 18.9s	remaining: 18.9s
500:	learn: 0.1307868	tota

644:	learn: 0.1031782	total: 27.5s	remaining: 15.1s
645:	learn: 0.1029710	total: 27.6s	remaining: 15.1s
646:	learn: 0.1027980	total: 27.6s	remaining: 15.1s
647:	learn: 0.1026316	total: 27.7s	remaining: 15s
648:	learn: 0.1024784	total: 27.8s	remaining: 15s
649:	learn: 0.1023594	total: 27.8s	remaining: 15s
650:	learn: 0.1022161	total: 27.8s	remaining: 14.9s
651:	learn: 0.1020329	total: 27.9s	remaining: 14.9s
652:	learn: 0.1018732	total: 27.9s	remaining: 14.8s
653:	learn: 0.1017452	total: 27.9s	remaining: 14.8s
654:	learn: 0.1016120	total: 28s	remaining: 14.7s
655:	learn: 0.1014777	total: 28s	remaining: 14.7s
656:	learn: 0.1014073	total: 28s	remaining: 14.6s
657:	learn: 0.1012761	total: 28.1s	remaining: 14.6s
658:	learn: 0.1010821	total: 28.1s	remaining: 14.5s
659:	learn: 0.1008648	total: 28.2s	remaining: 14.5s
660:	learn: 0.1008083	total: 28.2s	remaining: 14.5s
661:	learn: 0.1007585	total: 28.2s	remaining: 14.4s
662:	learn: 0.1005865	total: 28.2s	remaining: 14.4s
663:	learn: 0.1004905	to

807:	learn: 0.0822433	total: 33.3s	remaining: 7.91s
808:	learn: 0.0821195	total: 33.3s	remaining: 7.87s
809:	learn: 0.0819375	total: 33.4s	remaining: 7.82s
810:	learn: 0.0818016	total: 33.4s	remaining: 7.78s
811:	learn: 0.0816659	total: 33.4s	remaining: 7.73s
812:	learn: 0.0815378	total: 33.4s	remaining: 7.69s
813:	learn: 0.0814379	total: 33.5s	remaining: 7.64s
814:	learn: 0.0813847	total: 33.5s	remaining: 7.6s
815:	learn: 0.0812827	total: 33.5s	remaining: 7.56s
816:	learn: 0.0811879	total: 33.5s	remaining: 7.51s
817:	learn: 0.0809815	total: 33.6s	remaining: 7.47s
818:	learn: 0.0808393	total: 33.6s	remaining: 7.42s
819:	learn: 0.0806855	total: 33.6s	remaining: 7.38s
820:	learn: 0.0805559	total: 33.6s	remaining: 7.33s
821:	learn: 0.0804318	total: 33.7s	remaining: 7.29s
822:	learn: 0.0802966	total: 33.7s	remaining: 7.24s
823:	learn: 0.0801862	total: 33.7s	remaining: 7.2s
824:	learn: 0.0800562	total: 33.7s	remaining: 7.15s
825:	learn: 0.0799606	total: 33.8s	remaining: 7.11s
826:	learn: 0.

971:	learn: 0.0663400	total: 37.9s	remaining: 1.09s
972:	learn: 0.0663091	total: 37.9s	remaining: 1.05s
973:	learn: 0.0662430	total: 37.9s	remaining: 1.01s
974:	learn: 0.0661615	total: 37.9s	remaining: 973ms
975:	learn: 0.0660915	total: 38s	remaining: 934ms
976:	learn: 0.0660199	total: 38s	remaining: 895ms
977:	learn: 0.0659696	total: 38s	remaining: 856ms
978:	learn: 0.0658598	total: 38.1s	remaining: 817ms
979:	learn: 0.0658126	total: 38.1s	remaining: 778ms
980:	learn: 0.0657443	total: 38.1s	remaining: 739ms
981:	learn: 0.0657078	total: 38.2s	remaining: 700ms
982:	learn: 0.0656487	total: 38.2s	remaining: 661ms
983:	learn: 0.0655630	total: 38.2s	remaining: 621ms
984:	learn: 0.0654621	total: 38.2s	remaining: 582ms
985:	learn: 0.0653965	total: 38.3s	remaining: 543ms
986:	learn: 0.0653082	total: 38.3s	remaining: 504ms
987:	learn: 0.0652197	total: 38.3s	remaining: 465ms
988:	learn: 0.0651198	total: 38.3s	remaining: 426ms
989:	learn: 0.0650548	total: 38.4s	remaining: 388ms
990:	learn: 0.0650

,Model,Accuracy,Bal Acc.,Time
1,Extra Trees,55.247286,57.087023,6.816804
2,XGBoost,54.764777,50.873760,12.017451
3,LightGBM,54.282268,48.832730,5.445958
4,Skl HistGBM,53.437877,48.772949,17.479324
5,AdaBoost,52.593486,50.347956,5.713007
6,Skl GBM,51.990350,45.945829,35.582986
7,Random Forest,51.628468,55.152780,6.289297
8,CatBoost,51.387214,46.736065,42.817774
9,Decision Tree,31.724970,39.279413,4.351674


In [52]:
from sklearn.metrics import classification_report,confusion_matrix

print(confusion_matrix(y_test,pred))
print(classification_report(y_test, pred))

[[  8 112   6  23  15]
 [  2  19   6   7   0]
 [  2  12  55   6   1]
 [  0  28   0  16   0]
 [ 20  97  47  19 328]]
              precision    recall  f1-score   support

         Bus       0.25      0.05      0.08       164
         Car       0.07      0.56      0.13        34
       Still       0.48      0.72      0.58        76
       Train       0.23      0.36      0.28        44
     Walking       0.95      0.64      0.77       511

    accuracy                           0.51       829
   macro avg       0.40      0.47      0.37       829
weighted avg       0.70      0.51      0.56       829



In [53]:
import pickle

In [54]:
model = tree_classifiers["XGBoost"]
model.fit(x_train, y_train)
pred = model.predict(x_test) 

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:07:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [55]:
print(confusion_matrix(y_test,pred))
print(classification_report(y_test, pred))

[[ 21  92   8  22  21]
 [  3  16   4  11   0]
 [  1   8  60   5   2]
 [  0  22   0  22   0]
 [ 29  78  51  18 335]]
              precision    recall  f1-score   support

         Bus       0.39      0.13      0.19       164
         Car       0.07      0.47      0.13        34
       Still       0.49      0.79      0.60        76
       Train       0.28      0.50      0.36        44
     Walking       0.94      0.66      0.77       511

    accuracy                           0.55       829
   macro avg       0.43      0.51      0.41       829
weighted avg       0.72      0.55      0.59       829



In [56]:
# save the model to disk
filename = 'final_model_all.sav'
pickle.dump(model, open(filename, 'wb'))



In [57]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)

0.5476477683956574
